In [1]:
from naive_bayes import init, preprocesser, test_instances, split_into_train_test

dataset, features, continuous_features, target, possible_features_mrs2, possible_features_mrs3 = init()

dataset = preprocesser(dataset, target, continuous_features)

train_ds, test_ds = split_into_train_test(dataset,0.8)

print('Naive Bayes común')
print('m=1: ',test_instances(train_ds,test_ds,1,False))
print('m=10: ',test_instances(train_ds,test_ds,10,False))
print('m=100: ',test_instances(train_ds,test_ds,100,False))
print('m=1000: ',test_instances(train_ds,test_ds,1000,False))

print('Naive Bayes logarítmico')
print('m=1: ',test_instances(train_ds,test_ds,1,True))
print('m=10: ',test_instances(train_ds,test_ds,10,True))
print('m=100: ',test_instances(train_ds,test_ds,100,True))
print('m=1000: ',test_instances(train_ds,test_ds,1000,True))


Naive Bayes común
m=1:  80.37383177570094
m=10:  80.37383177570094
m=100:  63.55140186915887
m=1000:  24.065420560747665
Naive Bayes logarítmico
m=1:  80.37383177570094
m=10:  80.37383177570094
m=100:  63.55140186915887
m=1000:  24.065420560747665
